In [5]:
import os, json
from pathlib import Path
import sys
import polars as pl
rootdir = Path.cwd().parent.parent
sys.path.append(str(rootdir))

In [24]:
contracts = json.load(open(rootdir / "scripts/stablecoins/contracts.json"))

table_name = "logs"
for name, detail in contracts.items(): 
    address = detail["address"].lower()
    print(address)

    for table_name in ["logs", "transactions"]:

        file_path = rootdir / "data" / "etherscan_raw" / f"{detail['chain']}_{address}" / f"{table_name}.parquet"
        
        df = pl.scan_parquet(file_path)
        n_rows = df.unique().collect().shape[0]
        max_block = df.select(pl.col("blockNumber").max()).collect().item()
        min_block = df.select(pl.col("blockNumber").min()).collect().item()

        print("\t",table_name, n_rows, min_block, max_block)

        if table_name == "logs":
            primary_key = ["logIndex", "transactionHash"]
        elif table_name == "transactions":
            primary_key = ["transactionIndex", "hash"]
        n_rows_primary_key = df.unique(subset=primary_key).collect().shape[0]
        print("\t",table_name, n_rows_primary_key)

        assert n_rows_primary_key == n_rows


0xf939e0a03fb07f59a73314e73794be0e57ac1b4e
	 logs 1774710 17257952 23433008
	 logs 1774710
	 transactions 59407 17257952 23432857
	 transactions 59407
0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f
	 logs 641533 17698470 23433018
	 logs 641533
	 transactions 41339 17699326 23433001
	 transactions 41339
0xbc6da0fe9ad5f3b0d58160288917aa56653660e9
	 logs 141214 11937214 23432535
	 logs 141214
	 transactions 19385 11937214 23432488
	 transactions 19385
0x085780639cc2cacd35e474e71f4d000e2405d8f6
	 logs 126368 19287523 23432896
	 logs 126368
	 transactions 4085 19287523 23432251
	 transactions 4085
0xcacd6fd266af91b8aed52accc382b4e165586e29
	 logs 50406 21543360 23432924
	 logs 50406
	 transactions 4474 21632793 23432726
	 transactions 4474
0x865377367054516e17014ccded1e7d814edc9ce4
	 logs 398340 11915914 23433011
	 logs 398340
	 transactions 11646 11915875 23433001
	 transactions 11646
0x6440f144b7e50d6a8439336510312d2f54beb01d
	 logs 71958 22483043 23432921
	 logs 71958
	 transactions 2379 225

In [16]:
df.collect().head()

address,topics,data,blockNumber,blockHash,timeStamp,gasPrice,gasUsed,logIndex,transactionHash,transactionIndex,chainid,contract_address,chain
str,list[str],str,i64,str,i64,i64,i64,i64,str,i64,i64,str,str
"""0x6b175474e89094c44da98b954eed…","[""0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"", ""0x00000000000000000000000074de5d4fcbf63e00296fd95d33236b9794016631"", ""0x0000000000000000000000008cbf64cf5c7fdf2a9bde1c82d9e491aee2ca4bb7""]","""0x0000000000000000000000000000…",12707741,"""0x71584ac3ea076d21f8676de00542…",1624684699,13000000000,171598,78,"""0xc274fceaba165b988597db5e784e…",64,1,"""0x6b175474e89094c44da98b954eed…","""ethereum"""
"""0x6b175474e89094c44da98b954eed…","[""0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"", ""0x0000000000000000000000005d3a536e4d6dbd6114cc1ead35777bab948e3643"", ""0x0000000000000000000000000000000000000000000000000000000000000000""]","""0x0000000000000000000000000000…",10369332,"""0xf9a95be051b210dcb2762e3ea9da…",1593549247,24999310950,1547277,151,"""0x8119049649d97ef3b192b4fd65aa…",27,1,"""0x6b175474e89094c44da98b954eed…","""ethereum"""
"""0x6b175474e89094c44da98b954eed…","[""0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"", ""0x0000000000000000000000000000006daea1723962647b7e189d311d757fb793"", ""0x00000000000000000000000060594a405d53811d3bc4766596efd80fd545a270""]","""0x0000000000000000000000000000…",13474528,"""0xf14de195a8598edc0d6b1c3e2388…",1635002276,88183846236,121986,111,"""0x1d0560f04e8b484c8a3d65820717…",84,1,"""0x6b175474e89094c44da98b954eed…","""ethereum"""
"""0x6b175474e89094c44da98b954eed…","[""0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"", ""0x000000000000000000000000e3f277382419535245a345e923898c2d43f7cbe5"", ""0x000000000000000000000000bba4c8eb57df16c4cfabe4e9a3ab697a3e0c65d8""]","""0x0000000000000000000000000000…",13185012,"""0x9f7cfe6f2901b34aee9a4fccee66…",1631103040,98872000000,34730,39,"""0x7161f52167ffa2a3b0eadfbf2911…",48,1,"""0x6b175474e89094c44da98b954eed…","""ethereum"""
"""0x6b175474e89094c44da98b954eed…","[""0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"", ""0x00000000000000000000000056178a0d5f301baf6cf3e1cd53d9863437345bf9"", ""0x000000000000000000000000a478c2975ab1ea89e8196811f51a7b7ade33eb11""]","""0x0000000000000000000000000000…",11225693,"""0x28218199dfdcf90b509b0dfa5902…",1604956843,29000000001,114489,41,"""0x59c87595de60841ec16da1c4e16c…",22,1,"""0x6b175474e89094c44da98b954eed…","""ethereum"""
